In [1]:
import pandas as pd
import glob
import numpy as np
import datetime
import os

print(os.getcwd())
print(datetime.datetime.now())

/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/AgilOne/Offer_10_off_20_20Q4
2021-01-06 18:08:40.521657


In [2]:
list_initial_files=glob.glob("./InitialFiles/*.csv")
list_initial_files

['./InitialFiles/10-12 Month About to Lapse MS A_20201102075436.csv',
 './InitialFiles/Online Signup MS B_20201102074942.csv',
 './InitialFiles/13-18 Month Lapsed MS B_20201102073920.csv',
 './InitialFiles/Online Signup MS A_20201102074635.csv',
 './InitialFiles/10-12 Month About to Lapse MS B_20201102075837.csv',
 './InitialFiles/13-18 Month Lapsed MS A_20201102073523.csv']

In [3]:
def recursive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root,file)
list_delta_files=list(recursive_file_gen("./DeltaFiles/"))
list_delta_files.sort()
list_delta_files

['./DeltaFiles/File_20201119/13_Month_Lapsed_Delta__20201119021501.csv',
 './DeltaFiles/File_20201119/Near_Lapsing_Delta__20201119021243.csv',
 './DeltaFiles/File_20201119/Reactivation_Redeemers__20201119031841.csv',
 './DeltaFiles/File_20201119/Signup_NoPurchase_Delta__20201119021251.csv',
 './DeltaFiles/File_20201125/13_Month_Lapsed_Delta__20201125080533.csv',
 './DeltaFiles/File_20201125/Near_Lapsing_Delta__20201125080541.csv',
 './DeltaFiles/File_20201125/Reactivation_Redeemers__20201125082106.csv',
 './DeltaFiles/File_20201125/Signup_NoPurchase_Delta__20201125080540.csv',
 './DeltaFiles/File_20201202/13_Month_Lapsed_Delta__20201202080404.csv',
 './DeltaFiles/File_20201202/Near_Lapsing_Delta__20201202080444.csv',
 './DeltaFiles/File_20201202/Reactivation_Redeemers__20201202081559.csv',
 './DeltaFiles/File_20201202/Signup_NoPurchase_Delta__20201202080407.csv',
 './DeltaFiles/File_20201209/13_Month_Lapsed_Delta__20201209080749.csv',
 './DeltaFiles/File_20201209/Near_Lapsing_Delta__20

In [4]:
dict_size={}
for file in list_initial_files:
    df=pd.read_csv(file)
    basename=os.path.basename(file)
    dict_f={basename:df.shape[0]}
    dict_size.update(dict_f)

In [5]:
for file in list_delta_files:
    df=pd.read_csv(file)
    basename=os.path.basename(file)
    dict_f={basename:df.shape[0]}
    dict_size.update(dict_f)

In [6]:
df_summary_size=pd.DataFrame(dict_size,index=[0]).T.reset_index()
df_summary_size.columns=['file_name','rows']
df_summary_size['type']=np.where(df_summary_size['file_name'].apply(lambda x: "20201102" in x),"initial",
                                 df_summary_size['file_name'].apply(lambda x: x.split("__")[0].split("_")[-1] if "__" in x else "")+\
                                 "_"+
                                 df_summary_size['file_name'].apply(lambda x: x.split("__")[1][:8] if "__" in x else ""))
df_summary_size=df_summary_size.sort_values(['type',"file_name"])                               




In [7]:
def define_set(str_file_name):
    if "_Delta_" in str_file_name:
        if "13_Month_Lapsed_" in str_file_name:
            return "13-18 Lapsed"
        elif "Near_Lapsing_" in str_file_name:
            return "10-12 about to lapse"
        elif "Signup_NoPurchase_" in str_file_name:
            return "Online SUNP"
        else:
            return np.nan
    elif "Reactivation_Redeemers" in str_file_name:
            return "Redeemers"
    else:
        if "13-18 Month Lapsed" in str_file_name:
            return "13-18 Lapsed"
        elif "10-12 Month About to Lapse" in str_file_name:
            return "10-12 about to lapse"
        elif "Online Signup" in str_file_name:
            return "Online SUNP"
        else:
            return np.nan
        

In [8]:
df_summary_size['type']=np.where(df_summary_size['file_name'].apply(lambda x: " MS A" in x),"Test",
                                 np.where(df_summary_size['file_name'].apply(lambda x: " MS B" in x),"Control",
                                          df_summary_size['type'])
                                )

df_summary_size['set']=df_summary_size['file_name'].apply(lambda x: define_set(x))

In [9]:
df_initial_audience=pd.DataFrame()
# list_delta_files_no_redeemers=[x for x in list_delta_files if "redeemer" not in x.lower()]
for file in list_initial_files:
    df=pd.read_csv(file,usecols=["customersummary_c_primaryscnhash"])
    df['segment']=os.path.basename(file)
    basename=os.path.basename(file)
    df_initial_audience=df_initial_audience.append(df)
print(df_initial_audience.shape)
df_initial_audience=df_initial_audience.drop_duplicates()
print(df_initial_audience.shape)

(10472244, 2)
(10472244, 2)


In [10]:
file_last_cumulative_redeemers=[x for x in list_delta_files if "redeemer" in x.lower()]
file_last_cumulative_redeemers.sort()
file_last_cumulative_redeemers=file_last_cumulative_redeemers[-1]
print(file_last_cumulative_redeemers)

df_latest_redeemers=pd.read_csv(file_last_cumulative_redeemers,usecols=["customersummary_c_primaryscnhash"])
df_latest_redeemers['redeem_label']="redeemers"
df_latest_redeemers=df_latest_redeemers[pd.notnull(df_latest_redeemers['customersummary_c_primaryscnhash'])]
df_latest_redeemers=df_latest_redeemers.drop_duplicates()
print(df_latest_redeemers.shape,df_latest_redeemers['customersummary_c_primaryscnhash'].nunique())
df_latest_redeemers.head(2)

./DeltaFiles/File_20210106/Reactivation_Redeemers__20210106082111.csv
(78116, 2) 78116


,customersummary_c_primaryscnhash,redeem_label
0,440aaa3a6079610e012e99c46a696cbfc13460bbb831de...,redeemers
1,78ed32303b71ce7c1e03010749e9e98f45577640a2d99c...,redeemers


In [11]:
df_latest_redeemers=pd.merge(df_latest_redeemers,df_initial_audience,on="customersummary_c_primaryscnhash",how="left")
df_latest_redeemers['segment']=df_latest_redeemers['segment'].fillna("others")
df_output_redeemers_summary=df_latest_redeemers.groupby("segment")['customersummary_c_primaryscnhash'].count().to_frame().reset_index()
df_output_redeemers_summary

,segment,customersummary_c_primaryscnhash
0,10-12 Month About to Lapse MS A_20201102075436...,25320
1,10-12 Month About to Lapse MS B_20201102075837...,100
2,13-18 Month Lapsed MS A_20201102073523.csv,16381
3,13-18 Month Lapsed MS B_20201102073920.csv,24
4,Online Signup MS A_20201102074635.csv,27448
5,Online Signup MS B_20201102074942.csv,1735
6,others,9246


In [12]:
df_output_redeemers_summary['customersummary_c_primaryscnhash'].sum()

80254

In [13]:
a=df_initial_audience['segment'].unique().tolist()
a.sort()
a

['10-12 Month About to Lapse MS A_20201102075436.csv',
 '10-12 Month About to Lapse MS B_20201102075837.csv',
 '13-18 Month Lapsed MS A_20201102073523.csv',
 '13-18 Month Lapsed MS B_20201102073920.csv',
 'Online Signup MS A_20201102074635.csv',
 'Online Signup MS B_20201102074942.csv']

In [14]:
writer=pd.ExcelWriter("./BL_10_off_20_size_JL_%s.xlsx"%str(datetime.datetime.now().date()),engine="xlsxwriter")
df_summary_size.to_excel(writer,"summary_delta_files",index=False)
df_output_redeemers_summary.to_excel(writer,"cum_redeemer_to_initial_aud",index=False)
writer.save()

In [15]:
datetime.datetime.now()

datetime.datetime(2021, 1, 6, 18, 10, 18, 765995)